In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import nba_api
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go 
import plotly.express as px

fig_DIR = "../figs/team_leaders/"

In [2]:
import os
os.environ["R_HOME"] = "C:\\Program Files\\R\\R-4.3.2\\"
%reload_ext rpy2.ipython

c:\Users\pansr\Documents\NBA\.venv\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
def get_df(stats):
    df = stats.get_data_frames()[0]
    df["Team"] = df["TEAM_NAME"]
    cols = df.columns
    df.columns = cols.str.replace("RATING","RTG")
    return df

In [4]:
from nba_api.stats.endpoints import leaguedashteamstats
stats = leaguedashteamstats.LeagueDashTeamStats(measure_type_detailed_defense="Advanced")
df = get_df(stats)
df = df.rename(columns={"OFF_RTG_RANK":"O_RANK","DEF_RTG_RANK":"D_RANK"})

In [5]:
df.columns

Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'E_OFF_RTG',
       'OFF_RTG', 'E_DEF_RTG', 'DEF_RTG', 'E_NET_RTG', 'NET_RTG', 'AST_PCT',
       'AST_TO', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT',
       'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE',
       'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'O_RANK',
       'D_RANK', 'NET_RTG_RANK', 'AST_PCT_RANK', 'AST_TO_RANK',
       'AST_RATIO_RANK', 'OREB_PCT_RANK', 'DREB_PCT_RANK', 'REB_PCT_RANK',
       'TM_TOV_PCT_RANK', 'EFG_PCT_RANK', 'TS_PCT_RANK', 'PACE_RANK',
       'PIE_RANK', 'Team'],
      dtype='object')

In [6]:
df1 = df.sort_values(by="NET_RTG",ascending=False)
df1 = df1.reset_index(drop=True)
df1 = df1.reset_index(drop=False)
df1["#"] = df1["index"] +1
df2 = df1[["#","Team","OFF_RTG","O_RANK","DEF_RTG","D_RANK","NET_RTG"]]
df = df2

In [7]:
n_colors = 30
colors = px.colors.sample_colorscale("Tropic", [n/(n_colors -1) for n in range(n_colors)])
orank = df["O_RANK"]-1
orank = orank.astype(np.int32).to_numpy()
carray_o =  np.array(colors)[orank]
drank = df["D_RANK"]-1
drank = drank.astype(np.int32).to_numpy()
carray_d =  np.array(colors)[drank]
nrank = df["#"]-1
nrank = nrank.astype(np.int32).to_numpy()
carray_n =  np.array(colors)[nrank]
title = "NBA Net Rating Leaders 2023-24"
font = dict(color="black",family="Arial Black, monospace", size=11)
fig = go.Figure(data=[go.Table(
        columnwidth=[5,50,15,15,15,15,15],
        header=dict(values=list(df.columns),
                    fill_color='snow',
                    align=['center','left','center','center'],
                    font=dict(color='Black',family="Arial Black, monospace", size=12),
                    line_color="grey"
                    ),
        cells=dict(values=[df["#"],df["Team"],df["OFF_RTG"],df["O_RANK"],df["DEF_RTG"],
                           df["D_RANK"],df["NET_RTG"]],
                fill_color=["white","white",carray_o,"white",carray_d,"white",carray_n],
                align=['center','left','center','center'],
                height=23,
                line_color="grey",#lightgrey",
                font=font,
                ),
                # height=25
        ),
    ])
    # fig.update_layout(title_text=title)
fig.update_layout(title=dict(text=title,y=0.99,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
fig.add_annotation(x=0.0, y=0.0,text="@SravanNBA",showarrow=False,xshift=1,yshift=5)
fig.add_annotation(x=1.0, y=0.0,text="Source: nba.com/stats",showarrow=False,xshift=1,yshift=5,font=dict(size=10))
# fig.update_layout(title=dict(text=title,y=0.01,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
tab_width = 550 
tab_height = 770
fig.update_layout(width=tab_width,height=tab_height,margin=dict(t=25,b=1,l=1,r=1))
# fig.update_layout(autosize=True)
fig.show()
fig.write_image(fig_DIR+f"Teams_NET_RTG.png", scale=3)

In [11]:
%%R -i df 
library(tidyverse)
library(gt)

df %>% 
  gt()%>%
  tab_header(
    title = md("**NBA Net Rating Leaders 2023-24**")) %>%
    data_color(columns = OFF_RTG, palette = c("red", "green")) %>%
    data_color(columns = DEF_RTG, palette = c("green","red")) %>%
    cols_align(align = "center",columns = c("#",OFF_RTG,O_RANK,DEF_RTG,D_RANK,NET_RTG))  %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% 
    gtsave("../figs/R/Teams_NET_RTG.png",zoom=5)

file:///C:/Users/pansr/AppData/Local/Temp/Rtmp2VEdsA/file5fd8151d74.html screenshot completed
